In [29]:
import pandas as pd
import os
import time
import requests

In [125]:
def fill_missing(series, limit):
    series = series.astype('str')
    series = ['0' + i if len(i) < limit else i for i in series]
    return series

In [129]:
def clean(df):
    
    df['origin_census_block_group'] = fill_missing(df['origin_census_block_group'], 12)
    df['county'] = [str(i)[0:5] for i in df['origin_census_block_group']]
    df['date_range_start'] = [str(i)[0:10] for i in df['date_range_start']]
    df['date_range_end'] = [str(i)[0:10] for i in df['date_range_end']]
    date_start = df.at[0, 'date_range_start']
    date_end = df.at[0, 'date_range_end']
    df = df.drop(['origin_census_block_group', 'bucketed_distance_traveled', 'median_dwell_at_bucketed_distance_traveled', 
                  'bucketed_home_dwell_time', 'at_home_by_each_hour', 'destination_cbgs', 'bucketed_away_from_home_time', 
                  'bucketed_percentage_time_home'], axis = 1)
    df = df.groupby('county').agg('median')
    df['date_range_start'] = [date_start] * len(df)
    df['date_range_end'] = [date_end] * len(df)
    df = df.reset_index()
    
    return df

In [157]:
start_time = time.time()

path = r'C:\Users\Andrew\Downloads\Data1'
dfs = []

for root, directories, files in os.walk(path, topdown=False):
    for name in files:
        dfs.append(clean(pd.read_csv(os.path.join(root, name))))

df = pd.concat(dfs, ignore_index = True)
df = df.drop(['mean_home_dwell_time', 'mean_non_home_dwell_time', 'mean_distance_traveled_from_home'], axis = 1)
df['state'] = [str(i)[0:2] for i in df['county']]

elapsed_time = time.time() - start_time
print(elapsed_time)

1541.4926471710205


In [182]:
def get_state_name(df):
    
    series = []
    
    url = 'https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt#:~:text=FIPS%20codes%20are%20numbers%20which,to%20which%20the%20county%20belongs.'
    page = requests.get(url)

    first = page.text.find('name\n   -----------   -------\n')
    second = page.text.find('name\n ------------    --------------\n')

    state = page.text[first + 37:second - 46].split('    ')
    state = [i for i in state if i != '']
    state = [i.replace('\n', '') for i in state]
    state = [i.replace('   ', '') for i in state]
    keys = state[::2]
    values = state[1::2]
    state = dict(zip(keys, values))

    count = 0

    for i in range(len(df)):
        
        try:
            series.append(state[str(df.at[count, 'state'])])
        except:
            series.append(None)

        count += 1
    
    return series

In [183]:
def get_county_name(df):
    
    series = []
    
    url = 'https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt#:~:text=FIPS%20codes%20are%20numbers%20which,to%20which%20the%20county%20belongs.'
    page = requests.get(url)

    first = page.text.find('name\n   -----------   -------\n')
    second = page.text.find('name\n ------------    --------------\n')

    county = page.text[second + 41:].split('    ')
    county = [i for i in county if i != '']
    county = [i for i in county if i.find('(') == -1]
    county = [i.replace('\n', '') for i in county]
    keys = county[::2]
    values = county[1::2]
    county = dict(zip(keys, values))

    count = 0

    for i in range(len(df)):
        
        try:
            series.append(county[str(df.at[count, 'county'])])
        except:
            series.append(None)

        count += 1
    
    return series

In [186]:
df['state_name'] = get_state_name(df)
df['county_name'] = get_county_name(df)
df.head()

,county,device_count,distance_traveled_from_home,completely_home_device_count,median_home_dwell_time,part_time_work_behavior_devices,full_time_work_behavior_devices,delivery_behavior_devices,median_non_home_dwell_time,candidate_device_count,median_percentage_time_home,date_range_start,date_range_end,county_name,state,state_name
0,01001,122.0,9227.5,34.0,933.0,11.5,4.0,3.0,82.5,196.0,88.5,2020-01-01,2020-01-02,Autauga County,01,ALABAMA
1,01003,172.0,9168.0,50.0,887.0,17.0,5.0,2.0,91.0,263.0,89.0,2020-01-01,2020-01-02,Baldwin County,01,ALABAMA
2,01005,61.0,7269.0,15.0,771.0,5.0,1.0,1.0,121.0,117.0,86.0,2020-01-01,2020-01-02,Barbour County,01,ALABAMA
3,01007,103.0,10470.0,29.0,933.0,8.0,2.0,3.0,89.0,178.0,89.0,2020-01-01,2020-01-02,Bibb County,01,ALABAMA
4,01009,151.0,10623.0,46.0,961.0,14.0,4.0,2.0,96.0,218.0,89.0,2020-01-01,2020-01-02,Blount County,01,ALABAMA


In [187]:
df.to_csv('DataSet1.csv', index = False)

In [176]:
def get_state_code(df):
    
    series = []
    
    url = 'https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt#:~:text=FIPS%20codes%20are%20numbers%20which,to%20which%20the%20county%20belongs.'
    page = requests.get(url)

    first = page.text.find('name\n   -----------   -------\n')
    second = page.text.find('name\n ------------    --------------\n')

    state = page.text[first + 37:second - 46].split('    ')
    state = [i for i in state if i != '']
    state = [i.replace('\n', '') for i in state]
    state = [i.replace('   ', '') for i in state]
    values = state[::2]
    keys = state[1::2]
    state = dict(zip(keys, values))

    count = 0

    for i in range(len(df)):
        
        try:
            series.append(state[str(df.at[count, 'State']).upper()])
        except:
            series.append(None)

        count += 1
    
    return series

In [188]:
def get_county_code(df):
    
    series = []
    
    url = 'https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt#:~:text=FIPS%20codes%20are%20numbers%20which,to%20which%20the%20county%20belongs.'
    page = requests.get(url)

    first = page.text.find('name\n   -----------   -------\n')
    second = page.text.find('name\n ------------    --------------\n')

    county = page.text[second + 41:].split('    ')
    county = [i for i in county if i != '']
    county = [i for i in county if i.find('(') == -1]
    county = [i.replace('\n', '') for i in county]
    values = county[::2]
    keys = county[1::2]
    county = dict(zip(keys, values))

    count = 0

    for i in range(len(df)):
        
        try:
            series.append(county[str(df.at[count, 'County']).upper()])
        except:
            series.append(None)

        count += 1
    
    return series

In [179]:
df = pd.read_csv('population.csv')
df['state_code']  = get_code(df)
df.head()

,rank,State,Pop,Growth,Pop2018,Pop2010,growthSince2010,Percent,density,state_code
0,1,California,39937500,0.0096,39557000,37320900,0.0701,0.1194,256.3728,06
1,2,Texas,29472300,0.0268,28701800,25242700,0.1676,0.0881,112.8204,48
2,3,Florida,21993000,0.0326,21299300,18845800,0.1670,0.0658,410.1259,12
3,4,New York,19440500,-0.0052,19542200,19400100,0.0021,0.0581,412.5218,36
4,5,Pennsylvania,12820900,0.0011,12807100,12711200,0.0086,0.0383,286.5454,42


In [181]:
df.to_csv('population.csv', index = False)